In [22]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.io import loadmat
from os.path import join as opj
import os
from scipy.stats import pearsonr

spm_file1 = r'/mnt/data/DCM/result_backup/2022.11.27/game1/separate_hexagon_2phases_correct_trials/Setall/6fold/sub-032/SPM.mat' # a part of run which have error trials.
spm_file2 = r'/mnt/data/DCM/result_backup/2022.11.8/game1/separate_hexagon_2phases_correct_trials/Setall/6fold/sub-032/SPM.mat'# all run

spmmat1 = loadmat(spm_file1,struct_as_record=False)
spmmat2 = loadmat(spm_file2,struct_as_record=False)

In [25]:
names1 = [i[0] for i in spmmat1['SPM'][0][0].xX[0][0].name[0]]
names2 = [i[0] for i in spmmat2['SPM'][0][0].xX[0][0].name[0]]

In [26]:
designMatrix1 = spmmat1['SPM'][0][0].xX[0][0].X
designMatrix2 = spmmat2['SPM'][0][0].xX[0][0].X

designMatrix1 = np.round(designMatrix1.astype('float32'),decimals=5)
designMatrix2 = np.round(designMatrix2.astype('float32'),decimals=5)

names1 = [i[0] for i in spmmat1['SPM'][0][0].xX[0][0].name[0]]
names2 = [i[0] for i in spmmat2['SPM'][0][0].xX[0][0].name[0]]

print((designMatrix1==designMatrix2).all())

from scipy.stats import pearsonr
r,p = pearsonr(designMatrix1.reshape(-1),designMatrix2.reshape(-1))
print(r,p)

<ipython-input-26-0a2e07d11000>:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  print((designMatrix1==designMatrix2).all())


AttributeError: 'bool' object has no attribute 'all'

In [14]:
designMatrix1 = spmmat1['SPM'][0][0].xX[0][0].nKX
designMatrix2 = spmmat2['SPM'][0][0].xX[0][0].nKX

designMatrix1 = np.round(designMatrix1.astype('float32'),decimals=5)
designMatrix2 = np.round(designMatrix2.astype('float32'),decimals=5)

names1 = [i[0] for i in spmmat1['SPM'][0][0].xX[0][0].name[0]]
names2 = [i[0] for i in spmmat2['SPM'][0][0].xX[0][0].name[0]]

print((designMatrix1==designMatrix2).all())
design_matrix1 = pd.DataFrame(designMatrix1, columns=names1)
design_matrix2 = pd.DataFrame(designMatrix2, columns=names2)

r,p = pearsonr(designMatrix1.reshape(-1),designMatrix2.reshape(-1))
print(r,p)

<ipython-input-14-718aab9df919>:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  print((designMatrix1==designMatrix2).all())


AttributeError: 'bool' object has no attribute 'all'

<ipython-input-31-8494b0a1d3b3>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  (designMatrix1==designMatrix2)


False

In [3]:
diff_matrix = designMatrix1 == designMatrix2
for i,c in zip(range(102),names1):
    sum_num = diff_matrix[:,i].sum()
    if sum_num != 980:
        print(c)
        print(sum_num)

<ipython-input-3-0b39043ddc27>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  diff_matrix = designMatrix1 == designMatrix2


TypeError: 'bool' object is not subscriptable

In [10]:
from misc.load_spm import SPMfile
spm_file = SPMfile('/mnt/workdir/DCM/BIDS/derivatives/Nipype/working_dir/game1/distance_spct/Setall/6fold/work_1st/_subj_id_031/level1conest/SPM.mat')

In [11]:
spm_file.design_matrix

,Sn(1) M1*bf(1),Sn(1) M2_corr*bf(1),Sn(1) M2_corrxdistance^1*bf(1),Sn(1) M2_error*bf(1),Sn(1) decision_corr*bf(1),Sn(1) decision_corrxdistance^1*bf(1),Sn(1) decision_error*bf(1),Sn(1) trans_x,Sn(1) trans_y,Sn(1) trans_z,...,Sn(6) rot_y,Sn(6) rot_z,Sn(6) csf,Sn(6) white_matter,Sn(1) constant,Sn(2) constant,Sn(3) constant,Sn(4) constant,Sn(5) constant,Sn(6) constant
0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,-0.035761,-0.560449,-0.295481,...,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0
1,0.089908,0.0,0.0,0.000000,0.0,0.0,0.000000,-0.037098,-0.490463,-0.238083,...,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0
2,0.615282,0.0,0.0,0.035692,0.0,0.0,0.000000,-0.034421,-0.500075,-0.222549,...,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0
3,0.540060,0.0,0.0,0.452444,0.0,0.0,0.000749,-0.030035,-0.473559,-0.154853,...,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0
4,0.151241,0.0,0.0,0.388090,0.0,0.0,0.169809,-0.032727,-0.317535,-0.103417,...,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,-0.000031,0.001445,-15.507247,-5.065361,0.0,0.0,0.0,0.0,0.0,1.0
976,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000263,0.001512,-2.013456,-7.029405,0.0,0.0,0.0,0.0,0.0,1.0
977,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000306,0.000942,-7.323095,16.875446,0.0,0.0,0.0,0.0,0.0,1.0
978,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,-0.000031,0.000487,0.398803,14.931156,0.0,0.0,0.0,0.0,0.0,1.0
